In [2]:
import os
from glob import glob
import xml.etree.ElementTree as xet
import random
from shutil import copy

In [3]:
data_len = len(glob('images/*.jpeg'))
print(data_len)

225


In [20]:
!mkdir data\train\images
!mkdir data\train\labels
!mkdir data\val\images
!mkdir data\val\labels

data_images = glob('images/*.jpeg')
data_xml_labels = glob('images/*.xml')

print(len(data_images))
print(len(data_xml_labels))

In [7]:
data_image_names = [file.split('\\')[-1].split('.')[0] for file in data_images]
# print(data_images_names)
data_label_names =  [file.split('\\')[-1].split('.')[0] for file in data_xml_labels]
print(set(data_image_names).difference(set(data_label_names)))

set()


In [14]:
def parseYOLOInfoFromXML(xml_path):
    tree = xet.parse(xml_path)
    root = tree.getroot()
    bbobj = root.find('object')
    bbinfo = bbobj.find('bndbox')

    size = root.find('size')
    img_width = int(size.find('width').text)
    img_height = int(size.find('height').text)
    
    xmin = int(bbinfo.find('xmin').text)
    ymin = int(bbinfo.find('ymin').text)
    xmax = int(bbinfo.find('xmax').text)
    ymax = int(bbinfo.find('ymax').text)

    box_width = xmax - xmin
    box_height = ymax - ymin

    norm_x_center = (xmin + box_width/2)/img_width
    norm_y_center = (ymin + box_height/2)/img_height
    norm_width = box_width/img_width
    norm_height = box_height/img_height
    
    return norm_x_center, norm_y_center, norm_width, norm_height

In [15]:
def createTXTFromXML(xml_path):
    norm_x_center, norm_y_center, norm_width, norm_height = parseYOLOInfoFromXML(xml_path)
    f_name = xml_path.split('\\')[-1].split('.')[0]
    text = f'0 {xmin} {ymin} {xmax} {ymax}' 
    with open(f'images/{f_name}.txt', 'w') as f:
        f.write(text)    

In [16]:
for xml_file in data_xml_labels:
    createTXTFromXML(xml_file)

In [17]:
## reorganizing after removing unwanted data

split_ratio = 0.8

source_folder = 'images'

data_images = glob('images/*.jpeg')
data_txt_labels = glob('images/*.txt')

data_shape = len(data_images)

In [18]:
random.shuffle(data_images)

In [21]:
data_images

['images\\N191.jpeg',
 'images\\N1.jpeg',
 'images\\N176.jpeg',
 'images\\N241.jpeg',
 'images\\N62.jpeg',
 'images\\N109.jpeg',
 'images\\N144.jpeg',
 'images\\N104.jpeg',
 'images\\N46.jpeg',
 'images\\N130.jpeg',
 'images\\N37.jpeg',
 'images\\N209.jpeg',
 'images\\N237.jpeg',
 'images\\N40.jpeg',
 'images\\N49.jpeg',
 'images\\N136.jpeg',
 'images\\N6.jpeg',
 'images\\N82.jpeg',
 'images\\N140.jpeg',
 'images\\N215.jpeg',
 'images\\N42.jpeg',
 'images\\N122.jpeg',
 'images\\N201.jpeg',
 'images\\N11.jpeg',
 'images\\N226.jpeg',
 'images\\N188.jpeg',
 'images\\N212.jpeg',
 'images\\N132.jpeg',
 'images\\N19.jpeg',
 'images\\N163.jpeg',
 'images\\N74.jpeg',
 'images\\N246.jpeg',
 'images\\N35.jpeg',
 'images\\N54.jpeg',
 'images\\N4.jpeg',
 'images\\N90.jpeg',
 'images\\N203.jpeg',
 'images\\N143.jpeg',
 'images\\N152.jpeg',
 'images\\N186.jpeg',
 'images\\N177.jpeg',
 'images\\N48.jpeg',
 'images\\N202.jpeg',
 'images\\N166.jpeg',
 'images\\N36.jpeg',
 'images\\N69.jpeg',
 'images\\

In [22]:
train_images = data_images[:int(split_ratio*data_shape)]
val_images = data_images[int(split_ratio*data_shape):]

print(len(train_images))
print(len(val_images))

180
45


In [23]:
train_file_names = [file.split('\\')[-1].split('.')[0] for file in train_images]
val_file_names = [file.split('\\')[-1].split('.')[0] for file in val_images]

In [24]:
train_labels = [f'images/{f_name}.txt' for f_name in train_file_names]
val_labels = [f'images/{f_name}.txt' for f_name in val_file_names]

In [25]:
# Moving the files into their respective folders

train_image_dir = 'data/train/images'
train_label_dir = 'data/train/labels'
val_image_dir = 'data/val/images'
val_label_dir = 'data/val/labels'

for idx, file in enumerate(train_images):
    copy(file, f'{train_image_dir}/{train_file_names[idx]}.jpeg')
for idx, file in enumerate(train_labels):
    copy(file, f'{train_label_dir}/{train_file_names[idx]}.txt')
for idx, file in enumerate(val_images):
    copy(file, f'{val_image_dir}/{val_file_names[idx]}.jpeg')
for idx, file in enumerate(val_labels):
    copy(file, f'{val_label_dir}/{val_file_names[idx]}.txt')